# Tutorial for AFMpy.SimAFM

## Imports

In [ ]:
# Standard library imports
import json
import logging
from pathlib import Path

# Import warnings for filtering out deprecation warnings
import warnings

# Filter the Bio warnings
warnings.filterwarnings('ignore', module='Bio')

# Third party imports
import MDAnalysis as MDA
from MDAnalysis import transformations
import matplotlib.pyplot as plt

# Filter the MDanalysis warnings
warnings.filterwarnings('ignore', module='MDAnalysis')

# AFMpy imports
from AFMpy import SimAFM, Stack, Plotting, Utilities

## Logging Configuration

Each module in AFMpy contains logging to for debugging purposes via the default python logging library. Logging for the modules should always be configured at the application level. Included in these tutorials are example logging configuration files that can be loaded with the following functions. You may adjust these logging configuration files as you see fit.

In [ ]:
# Load the preconfigured logging settings
with open('logs/SimAFM_Tutorial_LoggingConfig.json', 'r') as f:
    LOGGING_CONFIG = json.load(f)

# Set up the logging configuration
logging.config.dictConfig(LOGGING_CONFIG)

## Matplotlib Configuration

Included within the ```Plotting``` module are functions for creating the high quality figures. A default configuration that matches the figures in the publication is activated by running the following function.

In [ ]:
# Configure the rcParams for the plots
Plotting.configure_formatting()

## Load the Trajectory

The first step is to create an ```MDAnalysis.Universe``` object from our structure (PSF) and trajectory (DCD) files. We also apply a simple 90° rotation around the z-axis to orient the system in a visually pleasing way.

In [ ]:
# Define the paths to the PSF and DCD files
PSF_PATH = '../common/MD/Example_SecYEG.psf'
DCD_PATH = '../common/MD/Example_SecYEG_Aligned.dcd'

# Define a list of transformations to apply to the trajectory
# Create transformation workflow for periplasmic side.
rotz = transformations.rotateby(90, direction = [0,0,1], point = [0,0,0])
workflow = [rotz]

# Load the PSF and DCD files using MDAnalysis
universe = MDA.Universe(PSF_PATH, DCD_PATH, transformations = workflow)

## Set the Scanning Parameters

The shape of the simulated AFM tip is described by its radius in Å and its half-angle in degrees. Below, we set:
 - ```tip_radius = 20``` Å
 - ```tip_theta = 18``` degrees

Next, we decide the size and resolution of our AFM image by specifying:
 - ```boundaries = ((-64,64), (-64,64))``` (128 Å in each dimension)
 - ```scan_shape = (32,32)``` (Corresponds to a 4 Å/pixel resolution (128 Å / 32 pixels))

We then specify the atom selections to be scanned. Here, we scan all ```protein``` atoms whose z-position is above the average position of the phosphorous headgroup atoms in the membrane (```segid MEMB and name P```). Because the selection is evaluated each frame, if the protein fluctuates, the protruding portion is automatically updated before scanning.

Finally, since this data comes from an all-atom Molecular Dynamics simulation (CHARMM force field), we use ```AA_VDW_Dict()``` to map each atom’s van der Waals radius.

In [ ]:
# Tip shape parameters
tip_radius = 20
tip_theta = 18

# Scanning boundaries and shape
boundaries = ((-64,64),(-64,64))
scan_shape = (32,32)

# Atom Selections. Determine which atoms to scan, and which to consider the background.
# In this case, we scan the protein atoms, considering the heads of the lipids as the background.
protein_selection = 'protein'
membrane_selection = 'segid PHOS'
head_selection = 'name P'

# Choose the VDW radius mapping. In this case, we use the All Atom mapping.
vdw_mapping = SimAFM.AA_VDW_Dict()

## Run the AFM Simulation

After defining our simulation parameters and loading the universe, we create a grid of (x, y) points corresponding to the locations at which the AFM tip will scan. We then call ```simulate_AFM2D_stack``` to perform the simulated scans over each frame in the trajectory.

In [ ]:
# Create the grid of points to scan.
grid = SimAFM.make_grid(boundaries, scan_shape)

simulated_scans = SimAFM.simulate_AFM2D_stack(universe = universe,
                                              atom_selection = protein_selection,
                                              memb_selection = membrane_selection,
                                              head_selection = head_selection,
                                              grid = grid,
                                              tip_radius = tip_radius,
                                              tip_theta = tip_theta,
                                              vdw_dict = vdw_mapping)

## Create the Stack Object

Once the simulated scans have been created, the next step is to store them, along with additional metadata, in a single container that is compatible with the rest of the analysis/visualization package. Here, we create a ```Stack``` object, which holds both the data and associated information such as alignment details and resolution.

In [ ]:
# Create a dictionary to save the metadata of the scan.
metadata = {
    'Aligned': True,
    'Alignment Method': 'Transmembrane Backbone RMSD Minimization',
    'Side': 'Periplasmic',
    'PDB Code': '3DIN',
    'Membrane Composition': 'POPE',
    'Trajectory File': 'Example_SecYEG_Aligned.dcd',
    'Structure File': 'Example_SecYEG.psf',
    'Tip Radius': tip_radius,
    'Tip Theta': tip_theta
}

# Calculate the resolution from the grid
resolution = (boundaries[0][1] - boundaries[0][0]) / scan_shape[0]

# Create the stack object
stack = Stack.Stack(simulated_scans, resolution = resolution, **metadata)

## Plot the Simulated AFM image

Once the ```Stack``` object is created, we can visualize individual scans. In this example, we display the first scan in the dataset using the ```Stack.plot_image method```, which takes an image index and plots it on a specified Matplotlib axis. We use the custom ```LAFMcmap``` colormap from the ```Plotting``` module.

The ```Plotting``` module also provides helper functions for adding a scalebar and a colorbar: ```Plotting.add_scalebar``` and ```Plotting.add_colorbar```, respectively.

- Scalebar: ```Plotting.add_scalebar``` requires specifying the width of the scalebar in pixels, which supports subpixel values. Here, we set the width to ```10/stack.resolution``` (equivalent to 1 nm) and use a matching label.

- Colorbar: ```Plotting.add_colorbar``` automatically creates a colorbar axis, matching the intensity scale of the provided axis. We define its width as a fraction of the original axis width, specify the padding to the right of the axis, and add a label for the intensity scale.

In [ ]:
# Plot the first scan from the stack
fig, ax = plt.subplots(figsize = (6, 6))
ax.axis('off')
ax = stack.plot_image(0, ax = ax, cmap = Plotting.LAFMcmap)
ax = Plotting.add_scalebar(10/stack.resolution, label = '1nm')
cbar = Plotting.add_colorbar(width = '5%', pad = 0.08, label = 'Height (Å)')

plt.show()

## Save the Stack Object to a File

After generating the stack of AFM images, the final step is to store the results. We use a compressed pickle format (```.xz```) for efficient file size. However, unpickling data is inherently risky, as **maliciously crafted pickle files can execute arbitrary code.** Therefore, **never unpickle data from an untrusted source.**

To help mitigate this risk, we’ve included functions to cryptographically sign the pickle, ensuring data integrity when sharing. The process is straightforward:

 1. Generate or specify a private/public key pair. (You can use your own, or generate one on the fly.)
 2. Provide the private key to the ```Stack.save_compressed_pickle``` method. This creates an ```.xz``` file containing the pickle and a corresponding ```.sig``` file.
 3. Distribute the ```.xz```, ```.sig```, and ```.pub``` (public key) together. The private ```.pem``` file must never be shared.
 4. Recipients can then safely load the pickle with ```Stack.load_compressed_pickle```, verifying the signature using your public key.

If necessary, you can bypass signing/validation by setting ```sign_pickle``` or ```validate_pickle``` to ```False```. However, this practice is **not** recommended. Disabling signature checks leaves you vulnerable to malicious files—use it only at your own risk.

### Generate the Cryptographic Keys

In [ ]:
# Set the paths for the private and public keys we will use to digitally sign the stack.
PRIVATE_KEY_PATH = Path('keys/Tutorial_Private.pem')
PUBLIC_KEY_PATH = Path('keys/Tutorial_Public.pub')

# Create the keys directory if it does not exist.
PRIVATE_KEY_PATH.parent.mkdir(parents=True, exist_ok=True)

# If the keys do not exist, generate them.
if not PRIVATE_KEY_PATH.exists() or not PUBLIC_KEY_PATH.exists():
    Utilities.generate_keys(PRIVATE_KEY_PATH, PUBLIC_KEY_PATH)

### Save the Stack Object to a Pickle File

In [ ]:
# If the output directory does not exist, create it.
OUTPUT_DIR = Path('output')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Save the stack and digitally sign it with the private key.
stack.save_compressed_pickle(OUTPUT_DIR / 'Example_SecYEG_Stack.xz', private_key_filepath = PRIVATE_KEY_PATH)